In [1]:
from __future__ import print_function
from keras import optimizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.datasets import imdb
from keras.utils import np_utils
import numpy as np
import os
import random
from numpy import argmax
import string
import matplotlib.pyplot as plt
import io

Using TensorFlow backend.


In [2]:
base_dir = "C:\\Users\\samir.antonio\\Desktop\\patchs\\"

In [3]:
alphabet = string.printable

char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

# integer encode input data
def onehot_encoded_from_raw (data, char_to_int=char_to_int):
    integer_encoded = [char_to_int[char] for char in data]
    
    return onehot_encoded(integer_encoded, char_to_int)
    

# integer encode input data
def onehot_encoded (integer_encoded, char_to_int=char_to_int):
    # one hot encode
    onehot_encoded = list()
    for value in integer_encoded:
        letter = [0 for _ in range(len(char_to_int))]
        letter[value] = 1
        onehot_encoded.append(letter)
    
    return onehot_encoded

In [4]:
def add_left_padding (reviews_ints, seq_len = 30000):
    features = np.zeros((len(reviews_ints), seq_len), dtype=float)
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_len]
    return features

def load_data(train_porcentage):
    
    text_size = 30000

    labels = []
    data_list = []

    files = os.listdir(base_dir)
    
    result = 0
    
    for i in range(len(files)):
        file_path = base_dir + files[i]
        f = io.open(file_path, mode="r", encoding="utf-8")
        file = f.read()
        try:
            integer_encoded = [char_to_int[char]/len(alphabet) for char in file]
            #print(file_path)
            data_list.append(integer_encoded)
            commented = files[i].split('.')[0].split('-')[1]=='true'
            result_tmp = 1 if commented else 0
            if result_tmp != result:
                result = result_tmp
                labels.append(result)
        except:
            pass


    labels = np.asarray(onehot_encoded_from_raw(labels, [0,1]), dtype="int32")
    
    data = add_left_padding(data_list, text_size)
    
    tol_num = len(labels)
    
    train_num = int(tol_num*train_porcentage)
    
    rr = [i for i in range(tol_num)] 
    random.shuffle(rr)
    X_train = data[rr][:train_num]
    y_train = labels[rr][:train_num]
    X_test = data[rr][train_num:]
    y_test = labels[rr][train_num:]
    
    return (X_train,y_train),(X_test,y_test)

In [5]:
(x_train,y_train),(x_test,y_test) = load_data (0.9)

In [6]:
print(x_train.shape)
print(x_test.shape)
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

(523, 30000)
(59, 30000)


In [7]:

print('Build model...')
model = Sequential()
model.add(LSTM(10000, dropout=0.25, recurrent_dropout=0.25, return_sequences=False, input_shape=(1, 30000)))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Activation('sigmoid'))
model.add(Dense(2, activation='softmax'))

# try using different optimizers and different optimizer configs
#mean_squared_error
#binary_crossentropy
#model.compile(loss='mean_squared_error',
#              optimizer='adadelta',
#              metrics=['acc'])

#model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['acc'])

Build model...


In [8]:
batch_size = 10
epochs = 100

print('Train...')
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train...
Train on 523 samples, validate on 59 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[30000,40000]
	 [[Node: training/Adadelta/mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](training/Adadelta/mul/x, training/Adadelta/Variable/read)]]

Caused by op 'training/Adadelta/mul', defined at:
  File "C:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C:\Anaconda3\lib\site-packages\tornado\ioloop.py", line

In [ ]:
model.evaluate(x_test, y_test, verbose=0)


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
base_dir = "C:\\Users\\samir.antonio\\Desktop\\val\\patchs\\"

In [ ]:
(x_train,y_train),(x_test,y_test) = load_data (1)
print(x_train.shape)
print(x_test.shape)
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

In [ ]:
len(y_train)

In [ ]:
pred = model.predict(x_train)

In [ ]:
pred

In [ ]:
for j in range(len(pred)):
    real = y_test[j]
    if (real[1] == 1):
        print ('{} {}'.format(pred[j], real))

In [ ]:
model.evaluate(x_train, y_train)